# Model Coordinator

## ไลบรารี่ที่สำคัญ

- PySyft version 0.2.5
- Torch  version 1.4.0

In [1]:
import sys
import argparse
import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
import syft as sy
from syft import workers
#from syft.workers.websocket_client import WebsocketClientWorker

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/suparerk/opt/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


## เตรียมการประมวลผล

In [2]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
print('ประมวลผลโดยใช้ ' + str(device))

hook = sy.TorchHook(torch) 

ประมวลผลโดยใช้ cpu


## แบบจำลองโครงข่ายประสาทเทียม

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

# แบบจำลองเป็น Neural Network
model = Net()

# จำลองข้อมูลที่ฝั่ง Model_Coordinator เพื่อใช้สร้าง TrainConfig
# ไม่มีผลต่อการคำนวณ เนื่องข้อมูลอยู่ใน Data_Node
mock_data = torch.zeros(1, 2)

# สร้าง Traced_Model เป็น jit เพื่อให้สามารถส่งไปรันที่ Data_Node
traced_model = torch.jit.trace(model, mock_data)

# ใช้ Stochastic Gradient Descent
optimizer = "SGD"

# สร้าง Loss Function เป็น jit เพื่อให้สามารถส่งไปรันที่ Data_Node
@torch.jit.script
def loss_fn(target, pred):
    return ((target.view(pred.shape).float() - pred.float()) ** 2).mean()


# Hyperparameters
batch_size = 4
optimizer_args = {"lr" : 0.1, "weight_decay" : 0.01}
epochs = 1
max_nr_batches = -1
shuffle = True

# TrainConfig 
train_config = sy.TrainConfig(model=traced_model,
                              loss_fn=loss_fn,
                              optimizer=optimizer,
                              batch_size=batch_size,
                              optimizer_args=optimizer_args,
                              epochs=epochs,
                              shuffle=shuffle)

## การทดสอบโมเดลบน Model_Coordinator

In [6]:
data = torch.tensor([[0.0, 1.0], [1.0, 0.0], [1.0, 1.0], [0.0, 0.0]], requires_grad=True)
target = torch.tensor([[1.0], [1.0], [0.0], [0.0]], requires_grad=False)

print("\nEvaluation before training")
pred = model(data)
loss = loss_fn(target=target, pred=pred)
print("Loss: {}".format(loss))
print("Target: {}".format(target))
print("Pred: {}".format(pred))


Evaluation before training
Loss: 0.4933376908302307
Target: tensor([[1.],
        [1.],
        [0.],
        [0.]])
Pred: tensor([[ 0.1258],
        [-0.0994],
        [ 0.0033],
        [ 0.0210]], grad_fn=<AddmmBackward>)


## การส่งโมเดลไปยัง Data_Node

In [7]:
kwargs_websocket = {"host": "192.168.1.108", "hook": hook, "verbose": True}

data_node = sy.workers.websocket_client.WebsocketClientWorker(
    id="data_node", 
    port=8777, 
    **kwargs_websocket)

train_config.send(data_node)

## การเทรนบน Data_Node

In [8]:
for epoch in range(10):
    loss = data_node.fit(dataset_key="xor")
    print("-" * 50)
    print("Iteration %s: Data_Node's loss: %s" % (epoch, loss))

--------------------------------------------------
Iteration 0: Data_Node's loss: tensor(0.4933, requires_grad=True)
--------------------------------------------------
Iteration 1: Data_Node's loss: tensor(0.3484, requires_grad=True)
--------------------------------------------------
Iteration 2: Data_Node's loss: tensor(0.2858, requires_grad=True)
--------------------------------------------------
Iteration 3: Data_Node's loss: tensor(0.2626, requires_grad=True)
--------------------------------------------------
Iteration 4: Data_Node's loss: tensor(0.2529, requires_grad=True)
--------------------------------------------------
Iteration 5: Data_Node's loss: tensor(0.2474, requires_grad=True)
--------------------------------------------------
Iteration 6: Data_Node's loss: tensor(0.2441, requires_grad=True)
--------------------------------------------------
Iteration 7: Data_Node's loss: tensor(0.2412, requires_grad=True)
--------------------------------------------------
Iteration 8: 

## รับโมเดลที่ได้

In [9]:
new_model = train_config.model_ptr.get()

print("\nEvaluation after training:")
pred = new_model(data)
loss = loss_fn(target=target, pred=pred)
print("Loss: {}".format(loss))
print("Target: {}".format(target))
print("Pred: {}".format(pred))


Evaluation after training:
Loss: 0.23491761088371277
Target: tensor([[1.],
        [1.],
        [0.],
        [0.]])
Pred: tensor([[0.6553],
        [0.3781],
        [0.4834],
        [0.4477]], grad_fn=<DifferentiableGraphBackward>)
